# Mahjong Classification

This notebook aim on aply transfer learning on vgg16 to Mahjong detection.

In [1]:
# import dependency

import os
import math
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard, LearningRateScheduler
import keras
import datetime
import time
import sys
from pathlib import Path

Using TensorFlow backend.


In [2]:
# dimensions of our images.
#img_width, img_height = 224, 224
#as most of our test pattern around 40*40 , lets try some smaller size
#reset as no batter performace in 43 classes
img_width, img_height = 80, 80 

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test/'
nb_train_samples = 10800
nb_validation_samples = 450
#nb_test_samples = 550

#nb_epoch = 100
#it seems it cound learn more
nb_epoch = 35
#nb_class = 34
# update for classify 42 pattern and backOnly
nb_class = 43

ts = time.time()
logName = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M_%S')

### records:
|name| abort reason|
|----|-----|
| 2017_03_09_14_17_47 | change code to take more number of samples |
| 2017_03_09_14_51_31 | computer needed to shutdown |

|graph name |settings remarks| img dimensions| nb train samples | nb validation samples | nb_epoch | num of classes|img color info|
|-----------|----------------|---------------|------------------|-----------------------|----------|---------------|------|
|2017_03_08_12_42_19  | no change  | 224x224  | 2000 | 100| 100| 43| RGB|
|2017_03_08_16_42_42  | remove BackOnly Class  | 224x224  | 2000 | 100 | 100 | 42 | RGB |
|2017_03_08_21_34_23  | gray scale of 1st train | 224x224  | 2000 | 100 | 100 | 43 | gray |
|2017_03_09_02_50_02  | gray scale of 2nd train | 224x224  | 2000 | 100 | 100 | 42 | gray |
|2017_03_09_11_46_43  | lower dimensions of 42 classes | 112x112 | 2000 | 100 | 100 | 42 | gray |
|2017_03_09_13_11_36  | lower dimensions of 43 classes | 112x112 | 2000 | 100 | 100 | 43 | gray |
|2017_03_09_14_17_47  | increase number of steps (aborted ) | 224x224 | 2000 | 100 | 150 | 43 | gray |
|2017_03_09_14_51_31  | fix the number of train data (aborted ) | 224x224 | 10800| 450 | 100 | 43 | gray |
|2017_03_10_22_47_14  | fine tune parameters base on above results | 80x80 | 10800 | 450 | 35 |43 | gray |
|2017_03_12_02_21_50  | try to print evaluation in each epoch (failed )| 80x80 | 10800 | 450 | 35 |43 | gray |
|  | try to print evaluation at tensorboardd in each epoch (failed )| 80x80 | 10800 | 450 | 35 |43 | gray |

### DO NOT change the code, just add block
as cannot reverse


weights= None Or imagenet -- use the pretrained info
include_top=False -- do not use the trained relationship between class and feature
Dropout -- persent that drop some sample such that the nodel would not overfeed
softmax -- find the probability for classification
-----------
adadelta-- the best method to find the global min of the loss for this situation
metrics=['accuracy']--the tensorbox will log the accuracy
------------
train_datagen -- param for generating imgs
--------
lrate -- learning rate
-----
nb_epoch -- number of steps
----
observation:
43 Vs 42 classes --> higher initial acc in 43 (as removed inbalaced a class, backOnly with high amount of data)
224 * 224 Vs 112 * 112 img size ???? performace batter in 42 classes, but worse in 43 classes
-----
faster R-CNN-- draw the bounding box-- paper-- std method

In [3]:

# build the VGG16 network 'imagenet'
model = Sequential()
vggLayer = VGG16(include_top=False, weights=None, input_shape=(img_width, img_height, 3))


model.add(vggLayer)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=nb_class, activation='softmax', name='fc2'))

In [4]:
#load the weight 
# load weights into new model
model.load_weights("weights/2017_03_10_22_47_14/weights-improvement-20-0.95.hdf5")

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
"""
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
"""
test_datagen = ImageDataGenerator(rescale=1./255)
"""
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=8,
        class_mode='categorical')
"""
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=8,
        class_mode='categorical')



# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.25
    drop = 0.25
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

tfBoard = keras.callbacks.TensorBoard(log_dir='./logs/'+logName+"/", histogram_freq=2, write_graph=True)
filepath="./weights/"+logName+"/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


callbacks_list = [checkpoint, tfBoard, lrate]


if not os.path.exists("./weights/"):
    os.makedirs("./weights/")   
if not os.path.exists('./logs/'):
    os.makedirs('./logs/')   
if not os.path.exists("./weights/"+logName):
    os.makedirs("./weights/"+logName)   
if not os.path.exists('./logs/'+logName):
    os.makedirs('./logs/'+logName)   
#if not os.path.exists('./logs/'+logName+"_test"):
#    os.makedirs('./logs/'+logName+"_test")

Found 2325 images belonging to 43 classes.


In [63]:
#code for printing the prediciton 
#predict_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)


test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=8,
        class_mode='categorical',
        shuffle=False
        )


predicted = model.predict_generator(test_generator, val_samples=591)

test_pos_class_dict = dict(zip(test_generator.class_indices.values(), test_generator.class_indices.keys()) )


with open("./predictedResult.csv",'a') as predictedCsv:
    predictedCsv.write( "fileName,Org Class,Predicted Class\n" )
    for idx, _ in enumerate(predicted):
        predictedCsv.write(  str(os.path.basename(imgfilenames[idx])) +","+ str(os.path.dirname(imgfilenames[idx]))+","+ str(test_pos_class_dict.get( predicted[idx].argmax()))+'\n')


score = model.evaluate_generator(test_generator, val_samples=591)
print ("loss: %.2f%%\n acc: %.2f%%" %( score[0]*100, score[1]*100) )


Found 591 images belonging to 43 classes.
loss: 29.67%
 acc: 94.08%


In [5]:
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        callbacks=callbacks_list,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)



model.save_weights("./weights/"+logName+"/final.hdf5")

print("please use tensorboard to view logs")

INFO:tensorflow:Summary name block1_conv1_W:0 is illegal; using block1_conv1_W_0 instead.
INFO:tensorflow:Summary name block1_conv1_b:0 is illegal; using block1_conv1_b_0 instead.
INFO:tensorflow:Summary name block1_conv2_W:0 is illegal; using block1_conv2_W_0 instead.
INFO:tensorflow:Summary name block1_conv2_b:0 is illegal; using block1_conv2_b_0 instead.
INFO:tensorflow:Summary name block2_conv1_W:0 is illegal; using block2_conv1_W_0 instead.
INFO:tensorflow:Summary name block2_conv1_b:0 is illegal; using block2_conv1_b_0 instead.
INFO:tensorflow:Summary name block2_conv2_W:0 is illegal; using block2_conv2_W_0 instead.
INFO:tensorflow:Summary name block2_conv2_b:0 is illegal; using block2_conv2_b_0 instead.
INFO:tensorflow:Summary name block3_conv1_W:0 is illegal; using block3_conv1_W_0 instead.
INFO:tensorflow:Summary name block3_conv1_b:0 is illegal; using block3_conv1_b_0 instead.
INFO:tensorflow:Summary name block3_conv2_W:0 is illegal; using block3_conv2_W_0 instead.
INFO:tenso

NameError: global name 'test_acc' is not defined

In [ ]:
keras.__version__
